### 도서 정보 추출

In [4]:
import tools.ProcessFiles as pre_b
excel = pre_b.get_excel_data(1) # i차 분석

### Lv2 파일명 변경, Lv3/4/5 폴더 분류

In [ ]:
import tools.ProcessFiles as pre_b
pre_b.move_jsons(1) # i차 분석

---

### 개별 처리
- 불필요 페이지 제거
- 오타
- 페이지 머리말/꼬리말

In [3]:
import tools.ProcessWorkbook.quizbook3 as quiz

# quiz.main(json_files[0])

## 전체 돌리기

In [ ]:
import os, json
from typing import List, Dict, Any
import tools.ProcessFiles as pf
import tools.ProcessLv2 as pl2

json_files = pf.get_filelist(2)
json_files

In [6]:
excel = pf.get_excel_data(2)

In [7]:
excel

,ISBN,도서명,출판일,코퍼스 1분류,코퍼스 2분류,비고,분류
관리번호,,,,,,,
SS0001,9791196259976,"보험, 진짜 필요할까? : 보험을 제대로 모르는 사람들을 위한 또박또박 보험가이드",2018-03-29,보험,기초,,Lv2
SS0002,9791197848629,5분에 만나는 표준약관 : 생명보험,2022-04-15,보험,실무,,Lv3/4
SS0003,9791197848612,5분에 만나는 표준약관 : 질병·상해보험(손해보험 회사용),2022-04-15,보험,실무,,Lv3/4
SS0004,9791198375902,주식 가치평가를 위한 작은 책,2023-07-20,증권,의견,,Lv3/4
SS0005,9791192312620,세계 인플레이션 시대의 경제지표,2023-10-19,금융,기초,시사,Lv3/4
...,...,...,...,...,...,...,...
SS0264,9788960543201,알면 돈되는 보험 상품 100% 활용법,2023-09-20,보험,기초,,Lv3/4
SS0265,9791198974433,2025 감정평가사 객관식 민법,2024-11-04,보험,수험서,감정평가사,Lv3/4
SS0266,9791198804082,2025 감정평가사 부동산학원론 객관식,2024-09-03,보험,수험서,감정평가사,Lv3/4


In [7]:
import json

# for j in json_files:
if True:
    j = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/data/FINAL/2C_0902/Lv2/SS0037/SS0037.json'
    INPUT_PATH = j
    BACKUP_PATH = INPUT_PATH + ".bak"

    with open(INPUT_PATH, "r", encoding="utf-8") as f:
        origin = json.load(f)

    with open(BACKUP_PATH, "w", encoding="utf-8") as f:
        json.dump(origin, f, ensure_ascii=False, indent=4)

    # id = excel[excel['ISBN'] == int(origin.get('file_id'))].index[0]
    # new = {
    #     'file_id': str(excel.loc[id, 'ISBN']),
    #     'title': excel.loc[id, '도서명'],
    #     'cat1_domain': excel.loc[id, '코퍼스 1분류'],
    #     'cat2_sub': excel.loc[id, '코퍼스 2분류'],
    #     'cat3_specific': excel.loc[id, '비고'],
    #     'pub_date': str(excel.loc[id, '출판일'])[:10],
    #     'contents': [],
    # }
    
    
    # for i in range(len(origin['contents'])):
        # contents = origin['contents'][i]
        # c = extract_qna(contents)
        # page_contents = remove_enter(contents['page_contents'])
        # contents['page_contents'] = page_contents
        # c = contents
        # print(contents)  
    new = pl2.fill_chapter(origin)
    new = pl2.merge_paragraphs(new)

    with open(INPUT_PATH, "w", encoding="utf-8") as f:
        json.dump(new, f, ensure_ascii=False, indent=4)

In [ ]:
# for js in json_files:
#     INPUT_PATH = js
#     BACKUP_PATH = INPUT_PATH + ".bak"

#     if not os.path.exists(INPUT_PATH):
#         raise FileNotFoundError(INPUT_PATH)

#     with open(INPUT_PATH, "r", encoding="utf-8") as f:
#         data = json.load(f)

#     with open(BACKUP_PATH, "w", encoding="utf-8") as f:
#         json.dump(data, f, ensure_ascii=False, indent=4)

#     contents: List[Dict[str, Any]] = data.get("contents", [])
#     data["contents"] = build_output(contents)

#     with open(INPUT_PATH, "w", encoding="utf-8") as f:
#         json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
for i, j in enumerate(json_files):
    if '276709608' in j:
        print(i, j)